<a href="https://colab.research.google.com/github/Vaibhav-Tyro/keyphrase-extraction-using-BERT/blob/main/Data_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" importing all the libraries for data loading"""

import os
import numpy as np
import random
import sys
import torch
from pytorch_pretrained_bert import BertTokenizer
import utils
class DataLoader(object):
  def__init(self, data_dir, bert_model_dir, params, token_pad_idx=0): 
    self.data_dir = data_dir
    self.batch_size = params.batch_size
    self.max_len = params.max_len
    self.device = params.device
    self.seed = params.seed
    self.token_pad_idx = 0

    tags=self.load_tags()
    self.tag2idx = {tag : idx for idx,tag in enumerate(tags)}
    self.idx2tag = {idx: tag for idx, tag in enumerate(tags)}
    params.tag2idx = self.tag2idx 
    params.idx2tag = self.idx2tag
    self.tag_pad_idx = self.tag2idx['0']

    self.tokenizer = BertTokenizer.from_pretrained(bert_model_dir, do_lower_case = True)

    def load_tags(self):
      tags = []
      file_path = os.path.join(self.data_dir, 'tags.txt')
      with open(file_path, 'r') as file:
        for tag in file:
          tags.append(tags.strip())
      return tags

      def load_sentence_tags(self, sentences_file, tags_file, d):
        """ loads sentences and tags from their corresponding files.
            maps  tokens and tags from their indicies and stores them in the provided dict d.
        """
        sentence = []
        tags = []

        with open(sentence_file, 'r') as file:
          for line in file:
            # replace each tag by it's index
            tokens = line.split()
            sentences.append(self.tokenizer.convert_tokens_to_ids(tokens))

        with open(tags_file, 'r') as file:
          for line in file:
            #replace each tag by it's index
             tag_seq = [self.tag2idx.get(tag) for tag in line.strip().split(' ')]
             tags.append(tag_seq)

        # checks to ensure there is a tag for each token 
        assert len(sentence) == len(tags)
        for i in range(len(sentences)):
            # print(sentences[i], tags[i])
          assert len(tags[i]) == len(sentences[i])

        # storing sentences and tags in dict d
        d['data'] = sentences
        d['tags'] = tags
        d['size'] = len(sentences)
    def load_data(self, data_type):
      """ loads the data for each type in types from data_dir.

      Args:
          data_type: (str) has one of 'train', 'val', 'test' depending on which data is required.
      Returns:
          data: (dict) contains the data with tags for each type in types.
      """
      data = {}

      if data_type in ['train', 'val', 'test']:
        sentence_file = os.path.join(self.data_dir, data_type, 'sentence.txt')
        tags_path = os.path.join(self.data_dir, data_type, 'tags.txt')   
        self.load_sentence_tags(sentences_file, tags_path, data)
      else:
          raise ValueError("data type not in ['train', 'val', 'test']")
      return data

    def data_iterator(self,data,shuffle=false):
      """Returns a generator that yields betches data with tags.

      Args:
          data: (dict) contain data which has keys 'data', 'tags' and 'size'
          shuffle: (bool) whether the data should be shuffled

      yields:
          batch_data: (tensor) shape: (batch_size, max_len)
          batch_tags: (tensor) shape: (batch_size, max_len)
      """

      # make a list that decides the order in which we go over the data-this avoids explicit shuffling of data
      order = list(range(data['size']))
      if shuffle:
        random.seed(self.seed)
        random.shuffle(order)
      
      # one pass over data
      for i in range(data['size']//self.batch_size):
        # fetch sentences and tags
        sentences = [data['data'][idx] for idx in order[i*self.batch_size:(i+1)*self.batch_size]]
        tags = [data['tags'][idx] for idx in order[i*self.batch_size:(i+1)*self.batch_size]]

        # batch length
        batch_len = len(sentences)

        # compute length of longest sentence in batch
        batch_max_len = max([len(s) for s in sentences])
        max_len = min(batch_max_len, self.max_len)

        # prepare a numpy array with the data, initialising the data with pad_idx
        batch_data = self.token_pad_idx*np.ones((batch_len, max_len))
        batch_tags = self.tag_pad_idx * np.ones((batch_len, max_len))

        #copy the data to the numpy array
        for j in range(batch_len):
          cur_len = len(sentences[j])
          if cur_len <= max_len:
            batch_data[j][:cur_len] = sentences[j][:max_len]
            batch_tags[j] = tags[j][:max_len]
          else:
            batch_data[j] = sentences[j][:max_len]
            batch_tags[j] = tags[j][:max_len]

        # since all data are indicies, we convert them to torch LongTensors
        batch_data = torch.tensor(batch_data, dtype=torch.long)
        batch_tags = torch.tensor(batch_tags, dtype=torch.long)

        # shifts tensors to GPU if available
        batch_data, batch_tags = batch_data.to(self.device), batch_tags.to(self.device)

        yield batch_data, batch_tags

      

     


SyntaxError: ignored